In [1]:
import numpy as np
a = np.array([1,2,3,4])
print(a)


[1 2 3 4]


In [28]:
import time

a = np.random.rand(1000000)
b = np.random.rand(1000000)

tic = time.time()
c = np.dot(a,b)
toc = time.time()

print(c)
print("vector: "+str(1000*(toc-tic))+"ms")

c=0
tic=time.time()
for i in range(1000000):
    c+=a[i]*b[i]
toc=time.time()
print(c)
print("loop: "+str(1000*(toc-tic))+"ms")

#向量化的好处

249700.75030671962
vector: 0.9980201721191406ms
249700.7503067112
loop: 498.66604804992676ms


In [ ]:
import numpy as np
u=np.exp(v)

In [17]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable(
    "v",initializer = tf.zeros_initializer()(shape=[1]))
    

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable(
    "v",initializer = tf.ones_initializer()(shape=[1]))
    
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))


with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))
        


[0.]
[1.]


In [3]:
import tensorflow as tf

g=tf.Graph()
a=tf.constant([1.0,2.0],name="a")
b=tf.constant([2.0,3.0],name="b")

result =tf.add(a,b,name="add")
#result = a+b 
print (result)

Tensor("add_2:0", shape=(2,), dtype=float32)


In [18]:
import tensorflow as tf

a=tf.constant([1.0,2.0],name="a")
b=tf.constant([2.0,3.0],name="b")

result = a + b
print (result)
sess = tf.Session()
print(sess.run(result))
sess.close()



Tensor("add_8:0", shape=(2,), dtype=float32)
[3. 5.]


In [21]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1))

x=tf.placeholder(tf.float32,shape=(1,2),name="input")
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

sess = tf.Session()
init_op = tf.initialize_all_variables()
sess.run(init_op)

#print(sess.run(y))

print(sess.run(y,feed_dict={x:[[0.7,0.9]]}))

[[-1.6523422]]


In [6]:
#向前传播过程1
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x=tf.constant([[0.7,0.9]])

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

sess.run(w1.initializer)
sess.run(w2.initializer)

print(sess.run(y))
sess.close()

[[3.957578]]


In [13]:
#向前传播过程2
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x=tf.constant([[0.7,0.9]])

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

init_op = tf.global_variables_initializer()#便捷的变量初始化
sess.run(init_op)

print(sess.run(y))
sess.close()

[[3.957578]]


In [15]:
#向前传播过程3
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1))

x = tf.placeholder(tf.float32, shape=(3,2), name="input")


a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

init_op = tf.global_variables_initializer()#便捷的变量初始化
sess.run(init_op)

print(sess.run(y,feed_dict={x:[[0.7,0.9],[0.1,0.4],[0.5,0.8]]}))
#通过变量x替代常量x，需提供x的取值
sess.close()

[[1.2749494 ]
 [0.43409672]
 [1.0565504 ]]


In [32]:
#完整神经网络样例

import tensorflow as tf

from numpy.random import RandomState

batch_size = 8

w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.placeholder(tf.float32, shape=(None,2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None,1), name='y-input')
#
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)
#
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)
Y = [[int(x1+x2<1)]for(x1,x2) in X]

with tf.Session() as sess:
    init_op=tf.initialize_all_variables()
    sess.run(init_op)
    print (sess.run(w1))
    print (sess.run(w2))
            

    STEPS=5000
    for i in range(STEPS):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        sess.run(train_step,
                 feed_dict={x:X[start:end],y_:Y[start:end]})
        if i %1000 ==0:
            total_cross_entropy = sess.run(
                cross_entropy,feed_dict={x:X,y_:Y})
            print("after %d training steps,cross entropy on all data is %g"%(i,total_cross_entropy))
            
    print (sess.run(w1))
    print (sess.run(w2))
        


[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
after 0 training steps,cross entropy on all data is 0.0674925
after 1000 training steps,cross entropy on all data is 0.0163385
after 2000 training steps,cross entropy on all data is 0.00907547
after 3000 training steps,cross entropy on all data is 0.00714436
after 4000 training steps,cross entropy on all data is 0.00578471
[[-1.9618275  2.582354   1.6820377]
 [-3.4681718  1.0698231  2.11789  ]]
[[-1.824715 ]
 [ 2.6854665]
 [ 1.418195 ]]


In [39]:
import tensorflow as tf

v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])

sess = tf.InteractiveSession()
print (tf.greater(v1,v2).eval())
print (tf.where(tf.greater(v1,v2),v1,v2).eval())
sess.close()

[False False  True  True]
[4. 3. 3. 4.]


In [42]:
#自定义损失函数

import tensorflow as tf

from numpy.random import RandomState

batch_size = 8


x = tf.placeholder(tf.float32, shape=(None,2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None,1), name='y-input')

w1 = tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w1)
#


loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y,y_),
                             (y-y_)*loss_more,
                             (y_-y)*loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)


rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)
Y = [[x1+x2+rdm.rand()/10.0-0.05]for(x1,x2) in X]

with tf.Session() as sess:
    init_op=tf.initialize_all_variables()
    sess.run(init_op)
   
    STEPS=5000
    for i in range(STEPS):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        sess.run(train_step,
                 feed_dict={x:X[start:end],y_:Y[start:end]})

    print (sess.run(w1))
        

[[1.0193471]
 [1.0428091]]
